# Quantum Graph Neural Network (QGNN) Tutorial
This notebook shows how to run the tiny QGNN example.

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import numpy as np
from app.graphs import generate_graph_dataset, graph_to_adjacency_and_features
from app.trainer import train_qgnn, train_classical_gnn, split_and_prepare
from app.plots import plot_embedding_projection, plot_performance_bar
import matplotlib.pyplot as plt

# Generate a tiny dataset
raw_graphs = generate_graph_dataset()
processed = []
for graph, label in raw_graphs:
    adjacency, features = graph_to_adjacency_and_features(graph)
    processed.append(((adjacency, features), label))

train_set, test_set = split_and_prepare(processed, test_ratio=0.5)

# Train both models for a few epochs
qgnn_model, hist_q = train_qgnn(train_set, num_layers=1, epochs=3, lr=0.1)
classical_model, hist_c = train_classical_gnn(train_set, hidden_dim=4, epochs=3, lr=0.1)

# Evaluate on the test set
embeddings = []
targets = []
for adjacency, features, label in test_set:
    prob_q, emb = qgnn_model.forward((adjacency, features))
    embeddings.append(emb.flatten())
    targets.append(label)

plot_embedding_projection(np.vstack(embeddings), targets, 'examples/notebook_embeddings.svg')
plot_performance_bar(hist_q['acc'][-1], hist_c['acc'][-1], 'examples/notebook_performance.svg')

print('Notebook run complete.')